In [1]:
import pandas as pd
from datetime import date, timedelta
from sqlalchemy import create_engine

engine = create_engine('mysql+pymysql://root:@localhost:3306/portfolio_development')
conpf = engine.connect()

engine = create_engine("mysql+pymysql://root:@localhost:3306/stock")
const = engine.connect()

engine = create_engine("sqlite:///c:\\ruby\\port_lite\\db\\development.sqlite3")
conlite = engine.connect()

today = date.today()
today

datetime.date(2022, 1, 5)

### Sell transactions

In [3]:
format_dict = {
               'qty':'{:,}','SHARES':'{:,}','available_qty':'{:,}','volbuy':'{:,}','volsell':'{:,}','volbal':'{:,}',
               'price':'{:,.2f}','PRICE':'{:,.2f}','max_price':'{:,.2f}','min_price':'{:,.2f}',
               'buy_target':'{:,.2f}','sell_target':'{:,.2f}','fee':'{:,.2f}','vat':'{:,.2f}',
               'net':'{:,.2f}','profit':'{:,.2f}','cost':'{:,.2f}','volume':'{:,.2f}','beta':'{:,.2f}',
               'Q4':'{:,.3f}','Q3':'{:,.3f}','Q2':'{:,.3f}','Q1':'{:,.3f}','DIVIDEND':'{:,.3f}',
               'percent':'{:,.2f}%','PERCENT':'{:,.2f}%','yearly':'{:,.2f}%','pct':'{:,.2f}%',
              }

In [ ]:
pd.read_sql_query('SELECT * FROM buy ORDER BY date DESC LIMIT 1', const)

In [ ]:
date = today.strftime('%Y-%m-%d')
date

In [ ]:
sqlIns = """
INSERT INTO buy 
VALUES('WHART','2022-01-05',10000,12.50,0,0,1,0,'3','A2')
"""
print(sqlIns)

In [ ]:
rp = const.execute(sqlIns)
rp.rowcount

In [ ]:
sql = '''
SELECT T.name, S.* , B.net AS cost, B.qty 
FROM sells S
JOIN buys B
ON buy_id = B.id
JOIN stocks T
ON B.stock_id = T.id
WHERE S.date = '%s'
ORDER BY S.id DESC LIMIT 1
'''
sql = sql % today
sells = pd.read_sql(sql, conpf)
sells.style.format(format_dict)

In [ ]:
names = sells['name']
name = names.to_string(index=False)
name

In [ ]:
sr_qty = sells['qty']
qty_str = sr_qty.to_string(index=False)
qty = int(qty_str) * -1
qty

In [ ]:
sql = """
SELECT * 
FROM buy 
WHERE name = '%s'
"""
sql = sql % name
print(sql)

buy = pd.read_sql(sql, const)
buy

In [ ]:
sqlUpd = '''
UPDATE buy
SET volbuy = volbuy + %s
WHERE name = "%s"'''
sqlUpd = sqlUpd % (qty, name)
print(sqlUpd)

In [ ]:
rp = const.execute(sqlUpd)
rp.rowcount

In [ ]:
sqlDel = '''
DELETE FROM buy
WHERE volbuy = 0'''
rp = const.execute(sqlDel)
rp.rowcount

### Buys transactions

### Update if same stock already existed

In [4]:
sql = '''
SELECT T.name, B.* 
FROM buys B
JOIN stocks T
ON B.stock_id = T.id
WHERE B.date = '%s'
ORDER BY id DESC
LIMIT 1
'''
sql = sql % today
buys = pd.read_sql(sql, conpf)
buys.style.format(format_dict)

,name,id,stock_id,date,qty,price,fee,vat,status,net,kind,chart
0,DOHOME,1089,187,2022-01-05,"4,000",22.90,189.61,13.27,Active,"91,802.88",DTD,None


In [5]:
names = buys['name']
name = names.to_string(index=False)
name

'DOHOME'

In [6]:
sr_qty = buys['qty']
qty_str = sr_qty.to_string(index=False)
qty = int(qty_str)
qty

4000

In [7]:
sr_price = buys['price']
price_str = sr_price.to_string(index=False)
price = float(price_str)
price

22.9

In [11]:
new_cost = 24.3
sqlUpd = '''
UPDATE buy
SET volbuy = volbuy + %s, price = %s
WHERE name = "%s"'''
sqlUpd = sqlUpd % (qty, new_cost, name)
print(sqlUpd)


UPDATE buy
SET volbuy = volbuy + 4000, price = 24.3
WHERE name = "DOHOME"


In [10]:
rp = const.execute(sqlUpd)
rp.rowcount

1

### Update table dividend in stock database for sells transactions

In [15]:
sql = """
SELECT * 
FROM dividend 
WHERE name = "%s" 
"""
sql = sql % name
print(sql)

dividend = pd.read_sql(sql, const)
dividend.style.format(format_dict)


SELECT * 
FROM dividend 
WHERE name = "DOHOME" 



,NAME,Q4,Q3,Q2,Q1,DIVIDEND,PRICE,PERCENT,SHARES,XDATE,PAIDDATE,KIND,ACTUAL
0,DOHOME,0.100,0.000,0.000,0.000,0.100,0.00,0.00%,"12,000.0",2021-05-21,2021-05-21,,0


In [13]:
sqlUpd = '''
UPDATE dividend
SET shares = shares + %s
WHERE name = "%s"'''
sqlUpd = sqlUpd % (qty, name)
print(sqlUpd)


UPDATE dividend
SET shares = shares + 4000
WHERE name = "DOHOME"


In [14]:
rp = const.execute(sqlUpd)
rp.rowcount

1

### Update stocks in PortLite

In [19]:
sql = """
SELECT * 
FROM stocks
WHERE name = "%s"
"""
sql = sql % name
print(sql)

df_stocks = pd.read_sql(sql, conlite)
df_stocks.style.format(format_dict)


SELECT * 
FROM stocks
WHERE name = "DOHOME"



,id,name,max_price,min_price,status,buy_target,sell_target,volume,beta,cost,qty,buy_spread,sell_spread,available_qty,bl,sh,reason,market
0,612,DOHOME,30.75,12.46,I,22.90,26.75,138.29,0.88,24.30,"4,000",-9,16,"12,000",1,0,RD05%,SET100


In [17]:
cost = 24.3
sell_target = 26.75
status = 'I'
sqlUpd = """
UPDATE stocks
SET available_qty = available_qty + %s, cost = %s, sell_target = %s, status = '%s'
WHERE name = '%s'"""
sqlUpd = sqlUpd % (qty, cost, sell_target, status, name)
print(sqlUpd)


UPDATE stocks
SET available_qty = available_qty + 4000, cost = 24.3, sell_target = 26.75, status = 'I'
WHERE name = 'DOHOME'


In [18]:
rp = conlite.execute(sqlUpd)
rp.rowcount

1